In [4]:
import pandas as pd
import json

In [26]:
#Import recipes
recipes = pd.read_json('recipes.json', lines=True)
recipes.head()

,name,ingredients,url,image,cookTime,recipeYield,datePublished,prepTime,description
0,Easter Leftover Sandwich,12 whole Hard Boiled Eggs\n1/2 cup Mayonnaise\...,http://thepioneerwoman.com/cooking/2013/04/eas...,http://static.thepioneerwoman.com/cooking/file...,PT,8,2013-04-01,PT15M,Got leftover Easter eggs? Got leftover East...
1,Pasta with Pesto Cream Sauce,3/4 cups Fresh Basil Leaves\n1/2 cup Grated Pa...,http://thepioneerwoman.com/cooking/2011/06/pas...,http://static.thepioneerwoman.com/cooking/file...,PT10M,8,2011-06-06,PT6M,I finally have basil in my garden. Basil I can...
2,Herb Roasted Pork Tenderloin with Preserves,"2 whole Pork Tenderloins\n Salt And Pepper, to...",http://thepioneerwoman.com/cooking/2011/09/her...,http://static.thepioneerwoman.com/cooking/file...,PT15M,12,2011-09-15,PT5M,This was yummy. And easy. And pretty! And it t...
3,Chicken Florentine Pasta,"1 pound Penne\n4 whole Boneless, Skinless Chic...",http://thepioneerwoman.com/cooking/2012/04/chi...,http://static.thepioneerwoman.com/cooking/file...,PT20M,10,2012-04-23,PT10M,"I made this for a late lunch Saturday, and it ..."
4,Perfect Iced Coffee,"1 pound Ground Coffee (good, Rich Roast)\n8 qu...",http://thepioneerwoman.com/cooking/2011/06/per...,http://static.thepioneerwoman.com/cooking/file...,PT,24,2011-06-13,PT8H,"Iced coffee is my life. When I wake up, often ..."


In [90]:
#Extract every recipe containing chillies 
recipes_chilli = recipes[(recipes['ingredients'].str.contains('Chilies')) | (recipes['ingredients'].str.contains('Chili')) |
       (recipes['ingredients'].str.contains('Chiles')) | (recipes['ingredients'].str.contains('Chillies')) |
                        (recipes['ingredients'].str.contains('Chilie')) | (recipes['ingredients'].str.contains('Chile')) |
                         (recipes['ingredients'].str.contains('Chillie'))]
recipes_chilli.head()

,name,ingredients,url,image,cookTime,recipeYield,datePublished,prepTime,description
5,Easy Green Chile Enchiladas,"1 whole Onion, Diced\n2 Tablespoons Butter\n1 ...",http://thepioneerwoman.com/cooking/2012/05/eas...,http://static.thepioneerwoman.com/cooking/file...,PT10M,4,2012-05-31,PT5M,When I was in Albuquerque with Marlboro Man an...
23,Bacon-Wrapped Scallops with Chili Butter,"2 pounds Large Scallops\n1/2 pound Bacon, Cut ...",http://thepioneerwoman.com/cooking/2012/01/bac...,http://static.thepioneerwoman.com/cooking/file...,PT20M,8,2012-01-19,PT15M,"I made these beautiful, tasty little suckers b..."
25,Spicy Stewed Beef with Creamy Cheddar Grits,2 Tablespoons Canola Oil\n2 Tablespoons Butter...,http://thepioneerwoman.com/cooking/2012/01/spi...,http://static.thepioneerwoman.com/cooking/file...,PT3H,12,2012-01-30,PT20M,"My oh my, was this good. Ultra spicy, seriousl..."
30,Vegetarian Lettuce Wraps,2 teaspoons Peanut Or Olive Oil\n1 package (ab...,http://thepioneerwoman.com/cooking/2012/02/veg...,http://static.thepioneerwoman.com/cooking/file...,PT15M,6,2012-02-22,PT5M,"Today is Ash Wednesday, the beginning of Lent,..."
54,Grillin’ Recipe Contest Winners!,"4 whole Ribeye, Hanger, Flat Iron, Or Strip St...",http://thepioneerwoman.com/cooking/2012/05/gri...,http://static.thepioneerwoman.com/cooking/file...,PT5M,4,2012-05-17,PT10M,Before I share my fourth and final grilling re...


In [66]:
#Convert cookTime and prepTime into correct format
import isodate

recipes_chilli['cookTime_time'] = 0
recipes_chilli['prepTime_time'] = 0
for i in range(len(recipes_chilli)):
    recipes_chilli['cookTime_time'].iloc[i] = isodate.parse_duration(recipes_chilli['cookTime'].iloc[i])
    recipes_chilli['prepTime_time'].iloc[i] = isodate.parse_duration(recipes_chilli['prepTime'].iloc[i])
    
#Sum cooking and preparation time
recipes_chilli['total_time'] = recipes_chilli['cookTime_time'] + recipes_chilli['prepTime_time']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

In [84]:
#Create difficulty variable
import numpy as np

conditions = [
    (recipes_chilli['total_time'] > '01:00:00'),
    (('01:00:00' >= recipes_chilli['total_time']) & ( recipes_chilli['total_time'] >= '00:30:00')),
    ((recipes_chilli['total_time'] < '00:30:00'))
    ]

# create a list of the values we want to assign for each condition
values = ['Hard', 'Medium', 'Easy']

# create a new column and use np.select to assign values to it using our lists as arguments
recipes_chilli['difficulty'] = np.select(conditions, values)

recipes_chilli['difficulty'].fillna('Unknown', inplace = True)

# display updated DataFrame
recipes_chilli.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,name,ingredients,url,image,cookTime,recipeYield,datePublished,prepTime,description,cookTime_time,prepTime_time,total_time,difficulty
5,Easy Green Chile Enchiladas,"1 whole Onion, Diced\n2 Tablespoons Butter\n1 ...",http://thepioneerwoman.com/cooking/2012/05/eas...,http://static.thepioneerwoman.com/cooking/file...,PT10M,4,2012-05-31,PT5M,When I was in Albuquerque with Marlboro Man an...,0:10:00,0:05:00,00:15:00,Easy
23,Bacon-Wrapped Scallops with Chili Butter,"2 pounds Large Scallops\n1/2 pound Bacon, Cut ...",http://thepioneerwoman.com/cooking/2012/01/bac...,http://static.thepioneerwoman.com/cooking/file...,PT20M,8,2012-01-19,PT15M,"I made these beautiful, tasty little suckers b...",0:20:00,0:15:00,00:35:00,Medium
25,Spicy Stewed Beef with Creamy Cheddar Grits,2 Tablespoons Canola Oil\n2 Tablespoons Butter...,http://thepioneerwoman.com/cooking/2012/01/spi...,http://static.thepioneerwoman.com/cooking/file...,PT3H,12,2012-01-30,PT20M,"My oh my, was this good. Ultra spicy, seriousl...",3:00:00,0:20:00,03:20:00,Hard
30,Vegetarian Lettuce Wraps,2 teaspoons Peanut Or Olive Oil\n1 package (ab...,http://thepioneerwoman.com/cooking/2012/02/veg...,http://static.thepioneerwoman.com/cooking/file...,PT15M,6,2012-02-22,PT5M,"Today is Ash Wednesday, the beginning of Lent,...",0:15:00,0:05:00,00:20:00,Easy
54,Grillin’ Recipe Contest Winners!,"4 whole Ribeye, Hanger, Flat Iron, Or Strip St...",http://thepioneerwoman.com/cooking/2012/05/gri...,http://static.thepioneerwoman.com/cooking/file...,PT5M,4,2012-05-17,PT10M,Before I share my fourth and final grilling re...,0:05:00,0:10:00,00:15:00,Easy


In [85]:
#Save as csv file
recipes_chilli.to_csv('recipes_chilli.csv', header= True, index=False)